# Amazon Rekognition API Sample codes - Part1
Amazon Rekognition Blackbelt Part 1 基本編 でご紹介したAPIを試すノートブックです。

## 事前準備
### 必要なライブラリのインポート

In [ ]:
!pip install -U pip
!pip install -U boto3
!pip install -U awscli
!pip install -U IPython

In [ ]:
# AWS SDK for Python のインポート
import boto3
# 使用するライブラリのインポート
from IPython.display import display, Image, Video, HTML
import sys, json

In [ ]:
# Client APIの設定
rekognition = boto3.client('rekognition')
s3 = boto3.client('s3')

# Amazon Rekognition Image 

このサンプルノートブックでは、予め用意したAmazon S3バケット内の画像データを使用します。

In [ ]:
bucket = "rekognition-console-v4-prod-iad"

※ ご自身で用意した画像でAPIを試したい場合は、以下の2つのセルを実行してください。
セルの実行にはアップロード先となるS3バケットを指定する必要があります。Amazon S3バケットを新たに作成するか、既存のS3バケットのバケット名を指定してください。

In [ ]:
# Amazon S3バケットを指定してください
# 作成方法：https://docs.aws.amazon.com/ja_jp/AmazonS3/latest/userguide/create-bucket-overview.html
bucket = "<アップロード先となるバケット名を入力>"

# Amazon SageMakerをお使いの場合はこちらでも可能です
# import sagemaker
# bucket = sagemaker.Session().default_bucket()

In [ ]:
# 以下では"data/"フォルダ内のsample-picture.jpgをAmazon S3にアップロードする場合の例
# アップロード対象（ローカル）のパス
filename = 'data/sample-picture.jpg'
# Amazon S3のアップロード先のパス
object_name = 'sample-picture.jpg'

s3.upload_file(filename, bucket, object_name)

### ラベル検出
画像や動画内のラベル（物体やシーン、イベント、アクティビティ）を検出する機能
  - 検出された各ラベルは信頼スコア(%)と共に返される
  - 物体ラベルの場合は、その位置情報（境界ボックス）も返される

https://docs.aws.amazon.com/rekognition/latest/APIReference/API_DetectLabels.html

In [ ]:
bucket = "rekognition-console-v4-prod-iad"
photo =  "assets/StaticImageAssets/SampleImages/skateboard.jpg"
display(Image(url=s3.generate_presigned_url('get_object', Params={'Bucket': bucket, 'Key': photo}), width=500))

In [ ]:
response = rekognition.detect_labels(
    Image={'S3Object':{'Bucket':bucket,'Name':photo}},
    Features=["GENERAL_LABELS","IMAGE_PROPERTIES"],
    # MaxLabels=10,
    # MinConfidence=75,
    # Settings={"GeneralLabels": {"LabelInclusionFilters":["Cat"]},"ImageProperties": {"MaxDominantColors":10}}
)

In [ ]:
print('Detected labels for ' + photo)
print()
for label in response['Labels']:
    print("Label: " + label['Name'])
    print("Confidence: " + str(label['Confidence']))
    print("Instances:")

    for instance in label['Instances']:
        print(" Bounding box")
        print(" Top: " + str(instance['BoundingBox']['Top']))
        print(" Left: " + str(instance['BoundingBox']['Left']))
        print(" Width: " + str(instance['BoundingBox']['Width']))
        print(" Height: " + str(instance['BoundingBox']['Height']))
        print(" Confidence: " + str(instance['Confidence']))
        print()

    print("Parents:")
    for parent in label['Parents']:
        print(" " + parent['Name'])

    print("Aliases:")
    for alias in label['Aliases']:
        print(" " + alias['Name'])

        print("Categories:")
    for category in label['Categories']:
        print(" " + category['Name'])
        print("----------")
        print()

if "ImageProperties" in str(response):
    print("Quality:")
    print(response["ImageProperties"]["Quality"])
    print()
    print("DominantColors:")
    print(response["ImageProperties"]["DominantColors"])
    print()

# raw response
# print(response)

### 顔検出・分析
- 画像や動画内に含まれる人物の顔を検出し、検出された顔を分析する機能
    - 画像上の顔の位置・角度
    - ランドマーク座標（左/右目、口 など）
    - 顔属性（性別、年齢、感情 など)
    - などを検出
- 1 つの画像から最大 100 個検出可能

https://docs.aws.amazon.com/rekognition/latest/APIReference/API_DetectFaces.html

In [ ]:
bucket = "rekognition-console-v4-prod-iad"
photo =  "assets/StaticImageAssets/SampleImages/drive.jpg"
display(Image(url=s3.generate_presigned_url('get_object', Params={'Bucket': bucket, 'Key': photo}), width=500))

In [ ]:
response = rekognition.detect_faces(
    Image={'S3Object':{'Bucket':bucket,'Name':photo}},
    Attributes=['ALL']
)

In [ ]:
print('Detected faces for ' + photo)
for faceDetail in response['FaceDetails']:
    print('The detected face is between ' + str(faceDetail['AgeRange']['Low'])
          + ' and ' + str(faceDetail['AgeRange']['High']) + ' years old')

    print('Here are the other attributes:')
    print(json.dumps(faceDetail, indent=4, sort_keys=True))

    # Access predictions for individual face details and print them
    print("Gender: " + str(faceDetail['Gender']))
    print("Smile: " + str(faceDetail['Smile']))
    print("Eyeglasses: " + str(faceDetail['Eyeglasses']))
    print("Emotions: " + str(faceDetail['Emotions'][0]))
    
# raw response
# print(response)

###  顔の比較
- ある画像（ソース画像）内の顔と別の画像（ターゲット画像）内の複数の人物の顔を比較し類似度を測定
- ソース画像に含まれる 1 つの顔を、ターゲット画像内の最大 15 の顔と比較可能

https://docs.aws.amazon.com/rekognition/latest/APIReference/API_CompareFaces.html

In [ ]:
bucket = "rekognition-console-v4-prod-iad"
source_photo =  "assets/StaticImageAssets/SampleImages/source3.jpg"
target_photo =  "assets/StaticImageAssets/SampleImages/target3.jpg"
display(Image(url=s3.generate_presigned_url('get_object', Params={'Bucket': bucket, 'Key': source_photo}), width=500))
display(Image(url=s3.generate_presigned_url('get_object', Params={'Bucket': bucket, 'Key': target_photo}), width=500))

In [ ]:
response = rekognition.compare_faces(
    SourceImage={'S3Object': {"Bucket": bucket, "Name": source_photo}},
    TargetImage={'S3Object': {"Bucket": bucket, "Name": target_photo}},
    # SimilarityThreshold=80,
    # QualityFilter="NONE"
)

In [ ]:
for faceMatch in response['FaceMatches']:
    position = faceMatch['Face']['BoundingBox']
    similarity = str(faceMatch['Similarity'])
    print('The face at')
    print(" Bounding box")
    print(" Top: " + str(faceMatch['Face']['BoundingBox']['Top']))
    print(" Left: " + str(faceMatch['Face']['BoundingBox']['Left']))
    print(" Width: " + str(faceMatch['Face']['BoundingBox']['Width']))
    print(" Height: " + str(faceMatch['Face']['BoundingBox']['Height']))
    print(" Confidence: " + str(faceMatch['Face']['Confidence']))
    print()

    print('Here are the other attributes:')
    print(json.dumps(faceMatch, indent=4, sort_keys=True))

# raw response
# print(response)

### 有名人の認識
- スポーツやビジネス、メディアなど各業界の著名な人物の顔を認識
- 検出された有名人の名前や関連URL (IMDB や Wikidata など) を返す
- 「顔検出・分析」の場合と同様に画像上の顔の位置や、感情などの顔属性に関する情報も返される

https://docs.aws.amazon.com/rekognition/latest/APIReference/API_RecognizeCelebrities.html

In [ ]:
bucket = "rekognition-console-v4-prod-iad"
photo =  "assets/StaticImageAssets/SampleImages/andy_portrait.jpg"
display(Image(url=s3.generate_presigned_url('get_object', Params={'Bucket': bucket, 'Key': photo}),width=500))

In [ ]:
response = rekognition.recognize_celebrities(
    Image={'S3Object':{'Bucket':bucket,'Name':photo}}
)

In [ ]:
print('Detected faces for ' + photo)
for celebrity in response['CelebrityFaces']:
    print('Name: ' + celebrity['Name'])
    print('Id: ' + celebrity['Id'])
    print('KnownGender: ' + celebrity['KnownGender']['Type'])
    print('Smile: ' + str(celebrity['Face']['Smile']['Value']))
    print('Position:')
    print('   Left: ' + '{:.2f}'.format(celebrity['Face']['BoundingBox']['Height']))
    print('   Top: ' + '{:.2f}'.format(celebrity['Face']['BoundingBox']['Top']))
    print('Info')
    for url in celebrity['Urls']:
        print('   ' + url)
    print()

# raw response
# print(response)

### コンテンツのモデレーション
- 画像や動画内に不適切な内容が含まれているかどうかを判定
- 不適切コンテンツの種類は階層的な分類がなされており、ポリシーに合わせて利用可能（薬物はNGでタバコはOK、など）

https://docs.aws.amazon.com/rekognition/latest/APIReference/API_DetectModerationLabels.html

※ このノートブックでは不適切なコンテンツが含まれない画像を用いています。不適切なコンテンツを含む場合は、以下のセルのコメントアウトをはずしていただくかご自身で用意したデータでお試しください。

In [ ]:
bucket = "rekognition-console-v4-prod-iad"
photo =  "assets/StaticImageAssets/SampleImages/family_picnic.jpg"

# 不適切なコンテンツが含まれる場合を試す際はこちら
# photo =  "assets/StaticImageAssets/SampleImages/yoga_swimwear.jpg"

display(Image(url=s3.generate_presigned_url('get_object', Params={'Bucket': bucket, 'Key': photo}), width=500))

In [ ]:
response = rekognition.detect_moderation_labels(
    Image={'S3Object':{'Bucket':bucket,'Name':photo}},
    # MinConfidence=60
)

In [ ]:
print('Detected labels for ' + photo)
for label in response['ModerationLabels']:
    print (label['Name'] + ' : ' + str(label['Confidence']))
    print (label['ParentName'])
    
# raw response
print(response)

不適切なコンテンツが検出された場合は以下のような実行結果例となります。

Detected labels for xxxxxx.jpg
- Female Swimwear Or Underwear : 93.13668823242188
- Suggestive
- Suggestive : 93.13668823242188
- Revealing Clothes : 81.7039566040039
- Suggestive

{'ModerationLabels': [{'Confidence': 93.13668823242188, 'Name': 'Female Swimwear Or Underwear', 'ParentName': 'Suggestive'}, {'Confidence': 93.13668823242188, 'Name': 'Suggestive', 'ParentName': ''}, {'Confidence': 81.7039566040039, 'Name': 'Revealing Clothes', 'ParentName': 'Suggestive'}], 'ModerationModelVersion': '6.0', 'ResponseMetadata': {'RequestId': 'a345e986-0b20-406a-a365-ae3cf63cb8ea', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'a345e986-0b20-406a-a365-ae3cf63cb8ea', 'content-type': 'application/x-amz-json-1.1', 'content-length': '304', 'date': 'Fri, 14 Apr 2023 04:59:57 GMT'}, 'RetryAttempts': 0}}

### PPE（個人用保護具）検出
- 画像内の人物が着用している個人用保護具とその位置を検出
- 検出できる保護具は 3 種類
    - フェイスカバー（例：マスク）
    - ヘッドカバー　（例：ヘルメット）
    - ハンドカバー　（例：手袋）
    
https://docs.aws.amazon.com/rekognition/latest/APIReference/API_DetectProtectiveEquipment.html

In [ ]:
bucket = "rekognition-console-v4-prod-iad"
photo =  "assets/StaticImageAssets/SampleImages/ppe_group_updated.jpg"
display(Image(url=s3.generate_presigned_url('get_object', Params={'Bucket': bucket, 'Key': photo}),width=500))

In [ ]:
response = rekognition.detect_protective_equipment(
    Image={'S3Object': {'Bucket': bucket, 'Name': photo}},
    SummarizationAttributes={
        'MinConfidence': 80,
        'RequiredEquipmentTypes': ['FACE_COVER','HAND_COVER','HEAD_COVER']}
)

In [ ]:
def display_summary(summary_type, summary):
    print(summary_type + '\n\tIDs: ', end='')
    if (len(summary) == 0):
        print('None')
    else:
        for num, id in enumerate(summary, start=0):
            if num == len(summary) - 1:
                print(id)
            else:
                print(str(id) + ', ', end='')
                
print('Detected PPE for people in image ' + photo)
print('\nDetected people\n---------------')
for person in response['Persons']:

    print('Person ID: ' + str(person['Id']))
    print('Body Parts\n----------')
    body_parts = person['BodyParts']
    if len(body_parts) == 0:
        print('No body parts found')
    else:
        for body_part in body_parts:
            print('\t' + body_part['Name'] + '\n\t\tConfidence: ' + str(body_part['Confidence']))
            print('\n\t\tDetected PPE\n\t\t------------')
            ppe_items = body_part['EquipmentDetections']
            if len(ppe_items) == 0:
                print('\t\tNo PPE detected on ' + body_part['Name'])
            else:
                for ppe_item in ppe_items:
                    print('\t\t' + ppe_item['Type'] + '\n\t\t\tConfidence: ' + str(ppe_item['Confidence']))
                    print('\t\tCovers body part: ' + str(
                        ppe_item['CoversBodyPart']['Value']) + '\n\t\t\tConfidence: ' + str(
                        ppe_item['CoversBodyPart']['Confidence']))
                    print('\t\tBounding Box:')
                    print('\t\t\tTop: ' + str(ppe_item['BoundingBox']['Top']))
                    print('\t\t\tLeft: ' + str(ppe_item['BoundingBox']['Left']))
                    print('\t\t\tWidth: ' + str(ppe_item['BoundingBox']['Width']))
                    print('\t\t\tHeight: ' + str(ppe_item['BoundingBox']['Height']))
                    print('\t\t\tConfidence: ' + str(ppe_item['Confidence']))
        print()
    print()

print('Person ID Summary\n----------------')
display_summary('With required equipment', response['Summary']['PersonsWithRequiredEquipment'])
display_summary('Without required equipment', response['Summary']['PersonsWithoutRequiredEquipment'])
display_summary('Indeterminate', response['Summary']['PersonsIndeterminate'])


# raw response
# print(response)

### テキストの検出
- 画像や動画に含まれるテキストを検出
- 英語など複数の言語に対応（2023年3月現在日本語未対応）
- 画像の場合1枚あたり最大100単語、動画の場合1フレームあたり最大50単語検出可能

https://docs.aws.amazon.com/rekognition/latest/APIReference/API_DetectText.html

In [ ]:
bucket = "rekognition-console-v4-prod-iad"
photo =  "assets/StaticImageAssets/SampleImages/coffee_monday.jpg"
display(Image(url=s3.generate_presigned_url('get_object', Params={'Bucket': bucket, 'Key': photo})))

In [ ]:
response = rekognition.detect_text(
    Image={'S3Object': {'Bucket': bucket, 'Name': photo}}
)

In [ ]:
textDetections = response['TextDetections']
print('Detected text\n----------')
for text in textDetections:
    print('Detected text:' + text['DetectedText'])
    print('Confidence: ' + "{:.2f}".format(text['Confidence']) + "%")
    print('Id: {}'.format(text['Id']))
    if 'ParentId' in text:
        print('Parent Id: {}'.format(text['ParentId']))
    print('Type:' + text['Type'])
    print()

# raw response
# print(response)

# Amazon Rekognition Video


こちらのノートブックでは Amazon Rekognition Image にはないAmazon Rekognition Video のみにあるAPIを中心にご紹介します。
- 人物の動線検出
- ビデオセグメント検出
- ラベル検出（Optional：Amazon SNS トピックへの分析完了ステータスの発行）

上記以外の Amazon Rekognition Video API の詳細は以下を参照してください。
- 顔分析
  - [StartLabelDetection](https://docs.aws.amazon.com/rekognition/latest/APIReference/API_StartFaceDetection.html)
  - [GetFaceDetection](https://docs.aws.amazon.com/rekognition/latest/APIReference/API_GetFaceDetection.html)
- 有名人の認識
  - [StartCelecrityRecognition](https://docs.aws.amazon.com/rekognition/latest/APIReference/API_StartCelebrityRecognition.html)
  - [GetCelebrityRecognition](https://docs.aws.amazon.com/rekognition/latest/APIReference/API_GetCelebrityRecognition.html)
- コンテンツのモデレーション
  - [StartContentModeration](https://docs.aws.amazon.com/rekognition/latest/APIReference/API_StartContentModeration.html)
  - [GetContentModeration](https://docs.aws.amazon.com/rekognition/latest/APIReference/API_GetContentModeration.html)
- テキストの検出
  - [StartTextDetection](https://docs.aws.amazon.com/rekognition/latest/APIReference/API_StartTextDetection.html)
  - [GetTextDetection](https://docs.aws.amazon.com/rekognition/latest/APIReference/API_GetTextDetection.html)

### 事前準備 - 動画データをアップロードするAmazon S3バケットを指定
Amazon Rekognition Videoを使用するためには動画データを予めAmazon S3にアップロードしておく必要があります。

**動画に関しては、本ノートブックではサンプルデータは準備しておりませんので、ご自身でお持ちのmp4データやオープンソースの動画データセットなどを準備してください。**

ここではアップロード先となるS3バケットを指定します。Amazon S3バケットを新たに作成するか、既存のS3バケットのバケット名を指定してください。

In [ ]:
# Amazon S3バケットを指定してください
# 作成方法：https://docs.aws.amazon.com/ja_jp/AmazonS3/latest/userguide/create-bucket-overview.html
bucket = "<アップロード先となるバケット名を入力>"

# Amazon SageMakerをお使いの場合はこちらでも可能です
# import sagemaker
# bucket = sagemaker.Session().default_bucket()

## 人物の動線検出
動画における人物の動線を追跡する機能
- 検出された人物の位置や顔のランドマークに関する情報をタイムスタンプ付きで返す
- カメラが動いていても検出可能

詳細は以下をご参照ください
- https://docs.aws.amazon.com/ja_jp/rekognition/latest/dg/persons.html

API ドキュメント
- https://docs.aws.amazon.com/rekognition/latest/APIReference/API_StartPersonTracking.html
- https://docs.aws.amazon.com/rekognition/latest/APIReference/API_GetPersonTracking.html

In [ ]:
filename = '<アップロードしたいファイルのパスを入力>'
object_name = '< Amazon S3のアップロード先のパスを入力>'

s3.upload_file(filename, bucket, object_name)

In [ ]:
video_url = s3.generate_presigned_url('get_object', Params={'Bucket': bucket, 'Key': object_name})
videoTag = "<video controls='controls' autoplay width='640' height='360' name='Video' src='{0}'></video>".format(video_url)
videoui = "<table><tr><td style='vertical-align: top'>{}</td></tr></table>".format(videoTag)
display(HTML(videoui))

In [ ]:
start_person_tracking = rekognition.start_person_tracking(
    Video={
        'S3Object':{
            'Bucket': bucket,
            'Name': object_name
        }
    }
)
job_id = start_person_tracking['JobId']
display(start_person_tracking)

In [ ]:
import time
response = rekognition.get_person_tracking(
    JobId = job_id
)
print(response['JobStatus'])

while (response['JobStatus'] == 'IN_PROGRESS'):
    print('.', end='')
    time.sleep(10)
    response = rekognition.get_person_tracking(
        JobId = job_id
    )
display(response)

In [ ]:
print('Codec: ' + response['VideoMetadata']['Codec'])
print('Duration: ' + str(response['VideoMetadata']['DurationMillis']))
print('Format: ' + response['VideoMetadata']['Format'])
print('Frame rate: ' + str(response['VideoMetadata']['FrameRate']))
print()
            
for personDetection in response['Persons']:
    print('Index: ' + str(personDetection['Person']['Index']))
    print('Timestamp: ' + str(personDetection['Timestamp']))
    print("Bounding box")
    print(" Top: " + str(personDetection['Person']['BoundingBox']['Top']))
    print(" Left: " + str(personDetection['Person']['BoundingBox']['Left']))
    print(" Width: " + str(personDetection['Person']['BoundingBox']['Width']))
    print(" Height: " + str(personDetection['Person']['BoundingBox']['Height']))
    print()

## ビデオセグメント検出
動画における主要なセグメントを検出
- ブラックフレーム
- クレジット
- カラーバー
- スレート / スタジオロゴ
- ショット

詳細は以下のリンクをご参照ください
- https://docs.aws.amazon.com/ja_jp/rekognition/latest/dg/segments.html

API ドキュメント
- https://docs.aws.amazon.com/rekognition/latest/dg/API_StartSegmentDetection.html
- https://docs.aws.amazon.com/rekognition/latest/dg/API_GetSegmentDetection.html

In [ ]:
filename = '<アップロードしたいファイルのパスを入力>'
object_name = '< Amazon S3のアップロード先のパスを入力>'

s3.upload_file(filename, bucket, object_name)

In [ ]:
video_url = s3.generate_presigned_url('get_object', Params={'Bucket': bucket, 'Key': video})
videoTag = "<video controls='controls' autoplay width='640' height='360' name='Video' src='{0}'></video>".format(video_url)
videoui = "<table><tr><td style='vertical-align: top'>{}</td></tr></table>".format(videoTag)
display(HTML(videoui))

In [ ]:
start_segment_detection = rekognition.start_segment_detection(
    Video={
        'S3Object':{
            'Bucket': bucket,
            'Name': object_name
        }
    },
    SegmentTypes=["TECHNICAL_CUE", "SHOT"],
    Filters={
        "TechnicalCueFilter":{
            "BlackFrame": {
                        "MaxPixelThreshold": 0.1,
                        "MinCoveragePercentage": 60
                    },
            "MinSegmentConfidence": 80.0
        },
        "ShotFilter": {"MinSegmentConfidence": 80.0}
    }
)
job_id = start_segment_detection['JobId']
display(start_segment_detection)

In [ ]:
import time
response = rekognition.get_segment_detection(
    JobId = job_id
)
while (response['JobStatus'] == 'IN_PROGRESS'):
    print('.', end='')
    time.sleep(10)
    response = rekognition.get_segment_detection(
        JobId = job_id
    )
display(response)

In [ ]:
for segment in response['Segments']:
    if segment["Type"] == "TECHNICAL_CUE":
        print("Technical Cue")
        print(f"\tConfidence: {segment['TechnicalCueSegment']['Confidence']}")
        print(f"\tType: {segment['TechnicalCueSegment']['Type']}")

    if segment["Type"] == "SHOT":
        print("Shot")
        print(f"\tConfidence: {segment['ShotSegment']['Confidence']}")
        print(f"\tIndex: " + str(segment["ShotSegment"]["Index"]))

    print(f"\tDuration (milliseconds): {segment['DurationMillis']}")
    print(f"\tStart Timestamp (milliseconds): {segment['StartTimestampMillis']}")
    print(f"\tEnd Timestamp (milliseconds): {segment['EndTimestampMillis']}")

    print(f"\tStart timecode: {segment['StartTimecodeSMPTE']}")
    print(f"\tEnd timecode: {segment['EndTimecodeSMPTE']}")
    print(f"\tDuration timecode: {segment['DurationSMPTE']}")

    print(f"\tStart frame number {segment['StartFrameNumber']}")
    print(f"\tEnd frame number: {segment['EndFrameNumber']}")
    print(f"\tDuration frames: {segment['DurationFrames']}")

## ラベル検出 - Amazon SNS トピックへの分析完了ステータスの発行
Amazon Rekognition Video APIでは、オプションでAmazon Simple Notification Service (Amazon SNS) トピックを指定することで、ビデオの分析が完了した際に分析リクエストの完了通知をAmazon SNSトピックに発行することができます。トピック通知はJSON文字列であり、ジョブの識別子や完了ステータス（"SUCCEEDED", "FAILED", "ERROR"）などが含まれています。
Amazon SNSトピックに発行された分析完了のステータス情報は、AWS LambdaやAmazon Simple Queue Service (Amazon SQS) を用いて取得することができます。

以下では、Amazon Rekognition Video におけるラベル検出において、Amazon SNSトピックをオプションで設定し、Amazon SQSで完了ステータスを取得します。

#### Amazon SNS トピックとAmazon SQSキューの作成

In [ ]:
# エラーが出る場合はAmazon SNS および Amazon SQS 操作の権限があるかどうか確認してください
# https://docs.aws.amazon.com/ja_jp/rekognition/latest/dg/api-video-roles.html

sns = boto3.client('sns')        
sqs = boto3.client('sqs')

sns_topic_name="amazon-rekognition-notebook-topic"
topic_response=sns.create_topic(Name=sns_topic_name)
sns_topic_arn = topic_response['TopicArn']

sqs_queue_name="amazon-rekognition-notebook-queue"
sqs.create_queue(QueueName=sqs_queue_name)
sqs_queue_url = sqs.get_queue_url(QueueName=sqs_queue_name)['QueueUrl']

attribs = sqs.get_queue_attributes(QueueUrl=sqs_queue_url,AttributeNames=['QueueArn'])['Attributes']
sqs_queue_arn = attribs['QueueArn']

# Amazon SNS トピックに Amazon SQS キューをサブスクライブ
sns.subscribe(
    TopicArn=sns_topic_arn,
    Protocol='sqs',
    Endpoint=sqs_queue_arn
)

In [ ]:
# Amazon SNSにAmazon SQSキューへの書き込みを許可
queue_policy = """{{
    "Version":"2012-10-17",
    "Statement":[
        {{
            "Sid":"MyPolicy",
            "Effect":"Allow",
            "Principal" : {{"AWS" : "*"}},
            "Action":"SQS:SendMessage",
            "Resource": "{}",
            "Condition":{{
                "ArnEquals":{{
                  "aws:SourceArn": "{}"
                }}
            }}
        }}
    ]
}}""".format(sqs_queue_arn, sns_topic_arn)

response = sqs.set_queue_attributes(
    QueueUrl = sqs_queue_url,
    Attributes = {
        'Policy' : queue_policy
    }
)

#### 既存の Amazon SNS トピックへのアクセス権限の付与
- Amazon Rekognition VideoがAmazon SNSトピックに完了ステータスを発行するための権限（IAMロール）を作成します。
- マネジメントコンソールからIAM（Identity and Access Management) にアクセスし、以下のポリシーを持つAmazon Rekognitionの IAM ロールを作成します。トピックARNには上記のsns_topic_arnの返り値となる'arn:...'から始まる文字列を代入してください。ロールを作成したらARNをコピーしておいてください。詳細な手順はこちらをご覧ください。 https://docs.aws.amazon.com/ja_jp/rekognition/latest/dg/api-video-roles.html#api-video-roles-single-topics
```
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "sns:Publish"
            ],
            "Resource": <<上記で作成したsns_topic_arn>>
        }
    ]
}
```


In [ ]:
role_arn = "<上で作成したIAMロールのARNを入力>"

In [ ]:
# StartLabelDetection API 
video = 'videos/amazon-go.mp4'
start_label_detection = rekognition.start_label_detection(
    Video={
        'S3Object':{
            'Bucket': bucket,
            'Name': video
        }
    },
    NotificationChannel={
        'RoleArn': role_arn,
        'SNSTopicArn':  sns_topic_arn
    },
    Features=['GENERAL_LABELS']
)
job_id = start_label_detection['JobId']
display(start_label_detection)

#### Amazon SQSからジョブ完了ステータスを取得

In [ ]:
jobFound = False
succeeded = False

dotLine=0
while jobFound == False:
    sqsResponse = sqs.receive_message(QueueUrl=sqs_queue_url, MessageAttributeNames=['ALL'], MaxNumberOfMessages=10)
    if sqsResponse:
        if 'Messages' not in sqsResponse:
            if dotLine<40:
                print('.', end='')
                dotLine=dotLine+1
            else:
                print()
                dotLine=0    
            sys.stdout.flush()
            time.sleep(5)
            continue

        for message in sqsResponse['Messages']:
            notification = json.loads(message['Body'])
            rekMessage = json.loads(notification['Message'])
            print(rekMessage['JobId'])
            print(rekMessage['Status'])
            if rekMessage['JobId'] == job_id:
                print('Matching Job Found:' + rekMessage['JobId'])
                jobFound = True
                if (rekMessage['Status']=='SUCCEEDED'):
                    succeeded=True
            sqs.delete_message(QueueUrl=sqs_queue_url, ReceiptHandle=message['ReceiptHandle'])

In [ ]:
response = rekognition.get_label_detection(
        JobId = job_id
)
display(response)

In [ ]:
for labelDetection in response['Labels']:
    label=labelDetection['Label']

    print("Timestamp: " + str(labelDetection['Timestamp']))
    print("   Label: " + label['Name'])
    print("   Confidence: " +  str(label['Confidence']))
    print("   Instances:")
    for instance in label['Instances']:
        print ("      Confidence: " + str(instance['Confidence']))
        print ("      Bounding box")
        print ("        Top: " + str(instance['BoundingBox']['Top']))
        print ("        Left: " + str(instance['BoundingBox']['Left']))
        print ("        Width: " +  str(instance['BoundingBox']['Width']))
        print ("        Height: " +  str(instance['BoundingBox']['Height']))
    print ("   Parents:")
    for parent in label['Parents']:
        print ("      " + parent['Name'])
    print ()